# Objeto a buscar
CUADERNOS FORMA FRANCESA RAYADO DE 96 HASTA 100 HOJAS MEDIDAS +/-16 X 21.5 CM +/- 1 CM

In [1]:
import requests
import json

def search_in_bing(termino_busqueda):
    # URL de la API de Bing Search
    url = "https://api.bing.microsoft.com/v7.0/search"

    # Parámetros de la solicitud
    parametros = {'q': termino_busqueda, "count": 10}

    # Encabezados de la solicitud, incluyendo tu clave de API
    encabezados = {'Ocp-Apim-Subscription-Key': "{Bing key aca}'"}

    # Realizar la solicitud
    respuesta = requests.get(url, headers=encabezados, params=parametros)

    # Verificar si la solicitud fue exitosa
    if respuesta.status_code == 200:
      r = respuesta.json()
      return [page["url"] for page in r["webPages"]["value"] if len(page["url"].split("/"))>3]
    else:
      return "Error en la solicitud: " + respuesta.reason

In [2]:
# Término de búsqueda
termino_busqueda = 'Comprar CUADERNOS FORMA FRANCESA RAYADO DE 96 HASTA 100 HOJAS MEDIDAS +/-16 X 21.5 CM +/- 1 CM'

# Realizar la búsqueda
resultados = search_in_bing(termino_busqueda)

# Imprimir resultados

In [ ]:
resultados

['https://www.tiendanorma.com.mx/p/cuaderno-argollado-frances-cuadro-grande-ferrari-sf-4-100-hojas/',
 'https://lumen.com.mx/cuaderno-forma-francesa-norma-jean-book-raya-100-hojas-marca-norma-sku-14005001',
 'https://lumen.com.mx/cuaderno-profesional-norma-jean-book-raya-100-hojas-marca-norma-sku-14009002',
 'https://www.amazon.com.mx/Cuadernos-Estrella-0080-Libreta-Francesa/dp/B00VXXL53E',
 'https://lumen.com.mx/libreta-pegada-forma-francesa-scribe-clasico-raya-96-hojas-marca-scribe-sku-7357',
 'https://lumen.com.mx/cuaderno-forma-francesa-scribe-clasico-blanco-100-hojas-marca-scribe-sku-14010002',
 'https://oficimundo.com.mx/librosflorete.html',
 'https://www.mercadolibre.com.mx/norma-color-360-frances-100-hojas-opcional-1-materias-pack-x-4-216cm-x-16cm-norma-color/p/MLM22769469',
 'https://www.mercadolibre.com.mx/estrella-raya-0080-192-hojas-bond-unidad-x-1-215cm-x-16cm-estrella/p/MLM22673776',
 'https://listado.mercadolibre.com.mx/cuaderno-200-hojas-francesa']

In [3]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

def get_url_text(url):
  html = requests.get(url).content
  soup = BeautifulSoup(html, features="html.parser")

  # kill all script and style elements
  for script in soup(["script", "style"]):
      script.extract()    # rip it out

  # get text
  try:
    text = soup.body.get_text()[0:10000]
  except:
    print("page not accesible, defaulting empty text")
    text = ""

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)
  return text

In [5]:
def get_gpt4_json_response(html_content):
    api_key = "open_ai_key_aca"
    client = OpenAI(api_key=api_key)
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_format={"type": "json_object"},
        messages=[
            {"role": "user", "content": f"""From this Page Text({html_content[0:120000]}) , try to extract as json the price (adjust just to get a floating number) of the item search, do declare if the call is successful or not: {{ "success": "OK", {{"item": "Extracted item name", "item_count":"Extracted item number of units", "cost/unit": "Extracted item cost per unit", "cost": "Extracted item total cost"}} }}"""}
        ]
    )

    return json.loads(completion.choices[0].message.content)

In [6]:
from openai import OpenAI

In [15]:
def get_page_data(url):
  html_content = get_url_text(url)
  print(f"Tamano de Pagina...{str(len(html_content))}")
  json_response = get_gpt4_json_response(html_content)
  json_response["URL"] = url
  return (json_response)

In [16]:
def get_item_data(item):
  print(f"Buscando...{item}")
  termino_busqueda = f'''Comprar {item}'''
  resultados = search_in_bing(termino_busqueda)
  output = []
  print(f"Explorando {str(len(resultados))} resultados...")
  for url in resultados:
    print(f"Obteniendo pagina...{url}")
    page_data=get_page_data(url)
    output.append(page_data)
  return output

In [17]:
resultados[0]

'https://lumen.com.mx/cuaderno-forma-francesa-norma-jean-book-raya-100-hojas-marca-norma-sku-14005001'

In [18]:
len(resultados[0].split("/")[:3])

3

In [ ]:
output

{'success': 'OK',
 'item': 'Cuaderno Argollado Frances Cuadro Grande Ferrari SF',
 'item_count': '100 Hojas',
 'cost/unit': '52.50',
 'cost': '52.50'}

In [20]:
output_analysis = get_item_data("CUADERNOS FORMA FRANCESA RAYADO DE 96 HASTA 100 HOJAS MEDIDAS +/-16 X 21.5 CM +/- 1 CM")


Buscando...CUADERNOS FORMA FRANCESA RAYADO DE 96 HASTA 100 HOJAS MEDIDAS +/-16 X 21.5 CM +/- 1 CM
Explorando 10 resultados...
Obteniendo pagina...https://www.tiendanorma.com.mx/p/cuaderno-argollado-frances-cuadro-grande-ferrari-sf-4-100-hojas/
Tamano de Pagina...3050
Obteniendo pagina...https://lumen.com.mx/cuaderno-forma-francesa-norma-jean-book-raya-100-hojas-marca-norma-sku-14005001
Tamano de Pagina...8116
Obteniendo pagina...https://www.amazon.com.mx/Cuadernos-Estrella-0080-Libreta-Francesa/dp/B00VXXL53E
Tamano de Pagina...7998
Obteniendo pagina...https://lumen.com.mx/cuaderno-profesional-norma-jean-book-raya-100-hojas-marca-norma-sku-14009002
Tamano de Pagina...8116
Obteniendo pagina...https://oficimundo.com.mx/librosflorete.html
Tamano de Pagina...3486
Obteniendo pagina...https://listado.mercadolibre.com.mx/cuaderno-200-hojas-francesa
Tamano de Pagina...9364
Obteniendo pagina...https://www.mercadolibre.com.mx/estrella-raya-0080-192-hojas-bond-unidad-x-1-215cm-x-16cm-estrella/p/ML

In [21]:
output_analysis

[{'success': 'OK',
  'item': 'Cuaderno Argollado Frances Cuadro Grande Ferrari SF',
  'item_count': '100',
  'cost/unit': '52.50',
  'cost': '5250.00',
  'URL': 'https://www.tiendanorma.com.mx/p/cuaderno-argollado-frances-cuadro-grande-ferrari-sf-4-100-hojas/'},
 {'success': 'OK',
  'item': 'CUADERNO FORMA FRANCESA NORMA JEAN BOOK RAYA 100 HOJAS',
  'item_count': '1',
  'cost/unit': '74.00',
  'cost': '74.00',
  'URL': 'https://lumen.com.mx/cuaderno-forma-francesa-norma-jean-book-raya-100-hojas-marca-norma-sku-14005001'},
 {'success': 'OK',
  'item': 'Cuadernos Estrella 0080 Libreta 1/4 Forma Francesa Rayado',
  'item_count': '1',
  'cost/unit': '146.98',
  'cost': '146.98',
  'URL': 'https://www.amazon.com.mx/Cuadernos-Estrella-0080-Libreta-Francesa/dp/B00VXXL53E'},
 {'success': 'OK',
  'item': 'CUADERNO PROFESIONAL NORMA JEAN BOOK RAYA 100 HOJAS',
  'item_count': '1',
  'cost/unit': '110.00',
  'cost': '110.00 MXN',
  'URL': 'https://lumen.com.mx/cuaderno-profesional-norma-jean-book-

In [22]:
json.dump(output_analysis, open("items_market.json", 'w'))